# OHLCVS Prediction using Prophet

## Import libraries and setup constant variables

In [3]:
import pandas as pd 
import psycopg2
from prophet import Prophet
from matplotlib import pyplot

Importing plotly failed. Interactive plots will not work.


In [4]:
CONNECTION = "dbname=postgres user=postgres password=horus123 host=localhost port=15432"
SELECT_QUERY = "select time_bucket('5 min', time) as time_5min, last(closing_price, time) as closing_price\
    from ohlcvs where exchange='bittrex' and symbol='BTC-USD' and time < '2020-01-01'\
    group by time_5min order by time_5min asc limit 70000;"
SELECT_QUERY_1MIN = "select time, closing_price from ohlcvs where exchange='bittrex' and symbol='BTC-USD' and time < '2020-01-01' order by time asc limit 70000;"
COLUMN_NAMES = ["ds", "y"]

def psql_to_df(connection_string, query, column_names):
    conn = psycopg2.connect(connection_string)
    with conn:
        cur = conn.cursor()
        cur.execute(query)
        rows = cur.fetchall()
        df = pd.DataFrame(rows, columns=column_names)
        return df

## Query the dataset and format the datetime column

In [28]:
df = psql_to_df(CONNECTION, SELECT_QUERY, COLUMN_NAMES)

In [29]:
df['ds'] = df['ds'].dt.tz_localize(None)

In [30]:
df.shape

(60000, 2)

In [31]:
# Number of rows in test set and number of periods to predict
TEST_SET_ROW_IDX = 50000

In [32]:
# The last 10 rows of the intended training set
df.iloc[TEST_SET_ROW_IDX-10:TEST_SET_ROW_IDX]

,ds,y
49990,2019-06-23 13:50:00,10602.530
49991,2019-06-23 13:55:00,10602.535
49992,2019-06-23 14:00:00,10645.844
49993,2019-06-23 14:05:00,10649.055
49994,2019-06-23 14:10:00,10669.663
49995,2019-06-23 14:15:00,10661.179
49996,2019-06-23 14:20:00,10662.664
49997,2019-06-23 14:25:00,10641.390
49998,2019-06-23 14:30:00,10622.988
49999,2019-06-23 14:35:00,10672.337


## Create a naive model, fit some training data in

In [ ]:
model = Prophet()
metrics = model.fit(df.iloc[:TEST_SET_ROW_IDX])

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


In [ ]:
N_PRED_PERIODS = 1000
future = model.make_future_dataframe(periods=N_PRED_PERIODS, freq="min")

In [ ]:
future.tail()

In [ ]:
forecast = model.predict(future)

## Plot some graphs

In [ ]:
fig1 = model.plot(forecast)

In [ ]:
fig2 = model.plot_components(forecast)

## Compare true vs predicted values

In [ ]:
y_pred = forecast.iloc[TEST_SET_ROW_IDX:TEST_SET_ROW_IDX+N_PRED_PERIODS]['yhat']
y_true = df.iloc[TEST_SET_ROW_IDX:TEST_SET_ROW_IDX+N_PRED_PERIODS]['y']
pyplot.plot(y_true, label='Actual')
pyplot.plot(y_pred, label='Predicted')
pyplot.legend()
pyplot.show()